<a href="https://colab.research.google.com/github/MyPham1207/Python_for_ML/blob/main/Assignments/Classification/Classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

import numpy as np

Mounted at /content/drive


##Step 1: Load data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/School/Python_for_ML/Assignment_6/Data/Wine.csv')
# shuffle the data or else the data after split will be skewed
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
0,12.96,3.45,2.35,18.5,106,1.39,0.70,0.40,0.94,5.28,0.68,1.75,675,3
1,13.45,3.70,2.60,23.0,111,1.70,0.92,0.43,1.46,10.68,0.85,1.56,695,3
2,12.29,3.17,2.21,18.0,88,2.85,2.99,0.45,2.81,2.30,1.42,2.83,406,2
3,14.13,4.10,2.74,24.5,96,2.05,0.76,0.56,1.35,9.20,0.61,1.60,560,3
4,14.16,2.51,2.48,20.0,91,1.68,0.70,0.44,1.24,9.70,0.62,1.71,660,3


##Step 2: Split the data and standardization

In [ ]:
# Split the data
n_splits = 5
kf = KFold(n_splits)
train_index = []
test_index = []
for train, test in kf.split(data):
  train_index.append(train)
  test_index.append(test)

In [ ]:
train_data = []
test_data = []
for i in range(n_splits):
  train_data.append(data.iloc[train_index[i]])
  test_data.append(data.iloc[test_index[i]])

train_data[0].head()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
36,11.84,0.89,2.58,18.0,94,2.20,2.21,0.22,2.35,3.05,0.79,3.08,520,2
37,12.85,3.27,2.58,22.0,106,1.65,0.60,0.60,0.96,5.58,0.87,2.11,570,3
38,14.19,1.59,2.48,16.5,108,3.30,3.93,0.32,1.86,8.70,1.23,2.82,1680,1
39,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
40,13.78,2.76,2.30,22.0,90,1.35,0.68,0.41,1.03,9.58,0.70,1.68,615,3


In [ ]:
norm_train_data = []
norm_test_data = []
for i in range(n_splits): 
  scaler = StandardScaler()
  transformer = scaler.fit(train_data[i])
  train_temp = pd.DataFrame(transformer.transform(train_data[i]), columns=train_data[i].columns, index=train_data[i].index) 
  test_temp = pd.DataFrame(transformer.transform(test_data[i]), columns=test_data[i].columns, index=test_data[i].index)
  norm_train_data.append(train_temp)
  norm_test_data.append(test_temp)


##Step 4: Training and calculating accuracy (have PCA)

In [ ]:
col = list(data.columns)
X_tag = col[0:len(col) - 1]
y_tag = [col[len(col) - 1]]
y_tag

['Customer_Segment']

In [ ]:
def train_and_predict(norm_train_data, norm_test_data, n_splits, model, use_PCA=False, num_components=2):
  y_predict = []
  for i in range(n_splits):

    if use_PCA:
      pca = PCA(n_components=num_components)
      pca = pca.fit(norm_train_data[i][X_tag])
      X_train = pca.transform(norm_train_data[i][X_tag])
      X_test = pca.transform(norm_test_data[i][X_tag])
    else:
      X_train = norm_train_data[i][X_tag]
      X_test = norm_test_data[i][X_tag]

    y_train = np.ravel(norm_train_data[i][y_tag])
    y_train_round = [round(y,0) for y in y_train]

    y_pred = model.fit(X_train, y_train_round).predict(X_test)
    y_predict.append(y_pred)

  return y_predict

In [ ]:
def accuracyCalc(y_predict, norm_test_data, n_splits):
  accuracy = 0
  for i in range(n_splits):
    
    y_test = np.ravel(norm_test_data[i][y_tag])
    y_test_round = [round(y,0) for y in y_test]
  
    acc = accuracy_score(y_test_round,  y_predict[i])
    accuracy += acc
  accuracy = accuracy / n_splits * 100
  return accuracy

In [ ]:
out = open('/content/drive/MyDrive/School/Python_for_ML/Assignment_6/Data/Accuracy_with_and_without_PCA.txt', 'w')

##a.Gaussian Naive Bayes

In [ ]:
model = GaussianNB()
out.write('- Gaussian Naive Bayes: \n')
y_predict1 = train_and_predict(norm_train_data, norm_test_data, n_splits, model)
acc1 = accuracyCalc(y_predict1, norm_test_data, n_splits)
out.write('+ Accuracy without PCA: ' + str(acc1) + '\n')
y_predict2 = train_and_predict(norm_train_data, norm_test_data, n_splits, model, True, 10)
acc2 = accuracyCalc(y_predict2, norm_test_data, n_splits)
out.write('+ Accuracy with PCA: ' + str(acc2) + '\n')
out.write('\n')

1

##b.Logistic Regression

In [ ]:
model = LogisticRegression(random_state=0)
out.write('- Logistic Regression: \n')
y_predict1 = train_and_predict(norm_train_data, norm_test_data, n_splits, model)
acc1 = accuracyCalc(y_predict1, norm_test_data, n_splits)
out.write('+ Accuracy without PCA: ' + str(acc1) + '\n')
y_predict2 = train_and_predict(norm_train_data, norm_test_data, n_splits, model, True, 10)
acc2 = accuracyCalc(y_predict2, norm_test_data, n_splits)
out.write('+ Accuracy with PCA: ' + str(acc2) + '\n')
out.write('\n')

1

##c.K-Nearest Neighbors

In [ ]:
model = KNeighborsClassifier(n_neighbors=3)
out.write('- K-Nearest Neighbors: \n')
y_predict1 = train_and_predict(norm_train_data, norm_test_data, n_splits, model)
acc1 = accuracyCalc(y_predict1, norm_test_data, n_splits)
out.write('+ Accuracy without PCA: ' + str(acc1) + '\n')
y_predict2 = train_and_predict(norm_train_data, norm_test_data, n_splits, model, True, 10)
acc2 = accuracyCalc(y_predict2, norm_test_data, n_splits)
out.write('+ Accuracy with PCA: ' + str(acc2) + '\n')
out.write('\n')

1

##d.Support Vector Machine

In [ ]:
model = svm.SVC()
out.write('- Support Vector Machine: \n')
y_predict1 = train_and_predict(norm_train_data, norm_test_data, n_splits, model)
acc1 = accuracyCalc(y_predict1, norm_test_data, n_splits)
out.write('+ Accuracy without PCA: ' + str(acc1) + '\n')
y_predict2 = train_and_predict(norm_train_data, norm_test_data, n_splits, model, True, 10)
acc2 = accuracyCalc(y_predict2, norm_test_data, n_splits)
out.write('+ Accuracy with PCA: ' + str(acc2) + '\n')
out.write('\n')

1

##e.Decision Tree

In [ ]:
model = tree.DecisionTreeClassifier()
out.write('- Decision Tree: \n')
y_predict1 = train_and_predict(norm_train_data, norm_test_data, n_splits, model)
acc1 = accuracyCalc(y_predict1, norm_test_data, n_splits)
out.write('+ Accuracy without PCA: ' + str(acc1) + '\n')
y_predict2 = train_and_predict(norm_train_data, norm_test_data, n_splits, model, True, 10)
acc2 = accuracyCalc(y_predict2, norm_test_data, n_splits)
out.write('+ Accuracy with PCA: ' + str(acc2) + '\n')
out.write('\n')

1

##f.Random Forest

In [ ]:
model = RandomForestClassifier(max_depth=2, random_state=0)
out.write('- Random Forest: \n')
y_predict1 = train_and_predict(norm_train_data, norm_test_data, n_splits, model)
acc1 = accuracyCalc(y_predict1, norm_test_data, n_splits)
out.write('+ Accuracy without PCA: ' + str(acc1) + '\n')
y_predict2 = train_and_predict(norm_train_data, norm_test_data, n_splits, model, True, 10)
acc2 = accuracyCalc(y_predict2, norm_test_data, n_splits)
out.write('+ Accuracy with PCA: ' + str(acc2) + '\n')
out.write('\n')

1

In [ ]:
out.close()